# All material ©2019, Alex Siegman

---

## Welcome to Projects in Programming & Data Science. Today we're going to jump right in to the mix and leverage the CitiBike API to populate a MySQL database at regular intervals. Consider this your warm-up for the semester! 

## MySQL is an open-source, relational database management system. Remember, a relational database can be thought of as an Excel file that contains multiple tabs (sheets). The benefit of these databases is that they are ripe for querying, and SQL (Structured Query Language) is one of the preferred methods for querying these relational databases.

---

## MySQL Setup 

**To install MySQL, in your terminal:**

1. \> brew install mysql
2. \> unset TMPDIR
3. \> mysql_secure_installation
4. \> {enter your preferred password}
5. \> {skip all options presented}

**To run MySQL, in your terminal:**

1. \> mysql -u root -p
2. \> {enter password}
3. \> quit

**To set MySQL privileges:** 

1. \> mysql -u root -p
2. \> {enter password}
3. \> grant all privileges on *.* to 'username'@'localhost' identified by 'password' 

> _to change your password, use mysql> alter user 'user'@'localhost' identified by 'newpassword'_

4. \> flush privileges
5. \> quit

**To create a MySQL database:**

1. mysql> create database citibike 

## Now, let's check out the API we'll be working with:

### https://streamdata.io/developers/api-gallery/new-york-citibike-api/

In [ ]:
# first, let's request the json from the CitiBike API URL

import json 
import urllib.request

with urllib.request.urlopen("https://feeds.citibikenyc.com/stations/stations.json") as url:
    data = json.loads(url.read().decode())
    print(data)

In [ ]:
stations = data['stationBeanList'] # iterate through the json to find the station data

In [ ]:
import pandas as pd # we'll use pandas just to visualize our data, NOT to query it

df_stations = pd.DataFrame(stations)
df_stations.head() # check the first five station entries

# you'll note that it has set 'altitude' as our index; if we were working only in Pandas we'd probably want to 
# change that, but we can leave it alone for now

In [ ]:
import MySQLdb 

# MySQLdb is a way to interface with a MySQL database via a Python API 
# http://mysql-python.sourceforge.net/MySQLdb.html

In [ ]:
db = MySQLdb.connect(passwd="password",db="citibike") # connect to our db

In [ ]:
c = db.cursor() # set our cursor

In [ ]:
# create our table

c.execute("CREATE TABLE IF NOT EXISTS StationsData (station_id int, stationName varchar(250), availableDocks int, totalDocks int, latitude float, longitude float, statusValue varchar(250), statusKey int, availableBikes int, stAddress1 varchar(250), stAddress2 varchar(250), city varchar(250), postalCode varchar(250), location varchar(250), altitude varchar(250), testStation bool, lastCommunicationTime date, landMark varchar(250));")

c.close()
db.commit()

In [ ]:
from datetime import datetime # import the datetime library 

c = db.cursor() # set our cursor 

# below, we are going to populate our table using "INSERT IGNORE INTO"
# this allows us to avoid errors when we periodically update our database 

query_template = """INSERT IGNORE INTO StationsData(station_id, stationName, availableDocks, totalDocks, latitude, \
longitude, statusValue, statusKey, availableBikes, stAddress1, stAddress2, city, postalCode, location, altitude, \
testStation, lastCommunicationTime, landMark) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""

for entry in stations: # for every station entry in the json 
    station_id = int(entry['id']) # find and set station_id
    stationName = str(entry['stationName'])
    availableDocks = int(entry['availableDocks'])
    totalDocks = int(entry['totalDocks'])
    latitude = str(entry['latitude'])
    longitude = str(entry['longitude'])
    statusValue = str(entry['statusValue'])
    statusKey = int(entry['statusKey'])
    availableBikes = int(entry['availableBikes'])
    stAddress1 = str(entry['stAddress1'])
    stAddress2 = str(entry['stAddress2'])
    city = str(entry['city'])
    postalCode = str(entry['postalCode'])
    location = str(entry['location'])
    altitude = str(entry['altitude'])
    testStation = bool(entry['testStation'])
    lastCommunicationTime = entry['lastCommunicationTime']
    landMark = str(entry['landMark'])
                           
    print("Inserting Station:", station_id, stationName, availableDocks, totalDocks, latitude, longitude, statusValue, statusKey, availableBikes, stAddress1, stAddress2, city, postalCode, location, altitude, testStation, lastCommunicationTime, landMark) 
    query_parameters = (station_id, stationName, availableDocks, totalDocks, latitude, longitude, statusValue, statusKey, availableBikes, stAddress1, stAddress2, city, postalCode, location, altitude, testStation, lastCommunicationTime, landMark) 
   
    c.execute(query_template, query_parameters) # execute insert query and parameters

c.close()
db.commit()

In [ ]:
c = db.cursor() # set our cursor 

c.execute("SELECT * FROM StationsData LIMIT 5;") # look at the first five entries 
rows = c.fetchall()

print(rows)

c.close()
db.commit()

## Last but not least, let's set things up so that our database automatically updates every 15 seconds. 

In [ ]:
import time 
import json 
import urllib.request
from datetime import datetime

while True:
    
    with urllib.request.urlopen("https://feeds.citibikenyc.com/stations/stations.json") as url:
        data = json.loads(url.read().decode())

    stations = data['stationBeanList'] # iterate through the json to find the station data

    db = MySQLdb.connect(passwd="password",db="citibike") # connect to our db

    c = db.cursor() # set our cursor

    query_template = """INSERT IGNORE INTO StationsData(station_id, stationName, availableDocks, totalDocks, latitude, \
    longitude, statusValue, statusKey, availableBikes, stAddress1, stAddress2, city, postalCode, location, altitude, \
    testStation, lastCommunicationTime, landMark) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""

    for entry in stations: # for every station entry in the json 
        station_id = int(entry['id']) # find and set station_id
        stationName = str(entry['stationName'])
        availableDocks = int(entry['availableDocks'])
        totalDocks = int(entry['totalDocks'])
        latitude = str(entry['latitude'])
        longitude = str(entry['longitude'])
        statusValue = str(entry['statusValue'])
        statusKey = int(entry['statusKey'])
        availableBikes = int(entry['availableBikes'])
        stAddress1 = str(entry['stAddress1'])
        stAddress2 = str(entry['stAddress2'])
        city = str(entry['city'])
        postalCode = str(entry['postalCode'])
        location = str(entry['location'])
        altitude = str(entry['altitude'])
        testStation = bool(entry['testStation'])
        lastCommunicationTime = entry['lastCommunicationTime']
        landMark = str(entry['landMark'])
                           
        print("Inserting Station:", station_id, stationName, availableDocks, totalDocks, latitude, longitude, statusValue, statusKey, availableBikes, stAddress1, stAddress2, city, postalCode, location, altitude, testStation, lastCommunicationTime, landMark) 
        query_parameters = (station_id, stationName, availableDocks, totalDocks, latitude, longitude, statusValue, statusKey, availableBikes, stAddress1, stAddress2, city, postalCode, location, altitude, testStation, lastCommunicationTime, landMark) 
   
        c.execute(query_template, query_parameters) # execute insert query and parameters
 
    c.close()
    db.commit()
        
    c = db.cursor() # set our cursor

    c.execute("SELECT * FROM StationsData LIMIT 5;") # look at the first five entries 
    rows = c.fetchall()

    print(rows)
        
    time.sleep(15)

---

## I hope this has helped you find your coding legs! Next week we'll get back to descriptive analytics using Python and Pandas. For now, take time to refresh yourself on the content covered in "Introduction to Programming". 

## If you need a referesher on your SQL skills, check out the "Supplementary Info" directory in the class repo.